# Llamada a la API 

In [17]:
import requests
import pandas as pd

# URL ajustada para el dataset específico de la OCDE desde la API, incluyendo datos de empleo por nivel educativo
url = "https://sdmx.oecd.org/public/rest/data/OECD.CFE.EDS,DSD_REG_EDU@DF_EMP,2.0/all?startPeriod=2013&dimensionAtObservation=AllDimensions"

# Encabezado para solicitar el formato JSON
headers = {
    "Accept": "application/vnd.sdmx.data+json; charset=utf-8; version=1.0"
}

# Realizar la solicitud a la API con un tiempo de espera extendido en caso de demoras
response = requests.get(url, headers=headers, timeout=30)


# Verificación del Estado de la Solicitud

In [18]:
# Verificar el estado de la solicitud
if response.status_code == 200:
    print("Datos recibidos con éxito")
else:
    print("Error en la solicitud:", response.status_code)

Datos recibidos con éxito


# Comprobación del Dataset

In [23]:
if response.status_code == 200:
    data = response.json()  # Asignar el JSON de respuesta a la variable 'data'
    print("Datos recibidos con éxito")
else:
    print("Error en la solicitud:", response.status_code)


Datos recibidos con éxito


# Obtener los Nombres de las Columnas Directamente del Dataset

In [33]:
if response.status_code == 200:
    data = response.json()
    if 'structure' in data['data'] and 'dimensions' in data['data']['structure']:
        dimension_labels = [dim['name'] for dim in data['data']['structure']['dimensions']['observation']]
        print("Nombres de las columnas extraídas automáticamente:", dimension_labels)
    else:
        print("Estructura de dimensiones no encontrada en la respuesta.")


Nombres de las columnas extraídas automáticamente: ['Frequency of observation', 'Territorial level', 'Reference area', 'Territorial typology', 'Measure', 'Age', 'Sex', 'Education level', 'Statistical operation', 'Unit of measure', 'Time period']


# Extracción y Estructuración de Datos en un DataFrame

In [38]:
# Crear listas vacías para almacenar los datos estructurados
records = []

# Verificar que la estructura del JSON contenga observaciones y extraerlas
if 'dataSets' in data['data']:
    observations = data['data']['dataSets'][0]['observations']
    
    # Extraer los datos de observación y construir los registros en el DataFrame
    for key, value in observations.items():
        # Convertir la clave en índices de dimensiones
        indices = key.split(':')
        
        # Crear un diccionario para la observación actual con las dimensiones mapeadas
        record = {dimension_labels[i]: indices[i] for i in range(len(indices))}
        record["Value"] = value[0]  # El valor de la observación
        
        # Agregar el registro a la lista de registros
        records.append(record)

    # Convertir a un DataFrame de pandas
    df = pd.DataFrame(records)
    print("DataFrame creado con éxito")
else:
    print("Observaciones no encontradas en la respuesta.")


DataFrame creado con éxito


# Mostrar las Primeras Filas del DataFrame

In [41]:
# Mostrar las primeras filas del DataFrame
df.head()


,Frequency of observation,Territorial level,Reference area,Territorial typology,Measure,Age,Sex,Education level,Statistical operation,Unit of measure,Time period,Value
0,0,0,0,0,0,0,0,0,0,0,0,1.409
1,0,0,0,0,0,0,0,0,0,0,1,1.176
2,0,0,0,0,0,0,0,0,0,0,2,1.187
3,0,0,0,0,0,0,0,0,0,0,3,1.022
4,0,0,0,0,0,0,0,0,0,0,4,1.277


# Información General del DataFrame. Valores No Nulos

In [45]:
# Información general del DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215469 entries, 0 to 215468
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Frequency of observation  215469 non-null  object 
 1   Territorial level         215469 non-null  object 
 2   Reference area            215469 non-null  object 
 3   Territorial typology      215469 non-null  object 
 4   Measure                   215469 non-null  object 
 5   Age                       215469 non-null  object 
 6   Sex                       215469 non-null  object 
 7   Education level           215469 non-null  object 
 8   Statistical operation     215469 non-null  object 
 9   Unit of measure           215469 non-null  object 
 10  Time period               215469 non-null  object 
 11  Value                     215469 non-null  float64
dtypes: float64(1), object(11)
memory usage: 19.7+ MB


# Exportación de Datos a un Archivo CSV

In [53]:
# Guardar el DataFrame en un archivo CSV
df.to_csv("ocde_data_education_unfiltered.csv", index=False)
print("El DataFrame se ha guardado en el archivo 'ocde_data_education_unfiltered.csv'")


El DataFrame se ha guardado en el archivo 'ocde_data_education_unfiltered.csv'
